<a href="https://colab.research.google.com/github/Dulon18/Machine_Learning_project/blob/main/Tomato_Leaf_Disease_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Importing Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import keras
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions

In [ ]:
len(os.listdir("/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train"))

# **Dataset Preprocessing**

In [ ]:
train_datagen = ImageDataGenerator(zoom_range=0.5,shear_range=0.3,horizontal_flip=True,preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

In [ ]:
train =train_datagen.flow_from_directory(directory='/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train',
                                         target_size=(256,256),batch_size=32)



validation =val_datagen.flow_from_directory(directory='/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/valid',
                                            target_size=(256,256),batch_size=32)

In [ ]:
t_img,label = train.next()

In [ ]:
def plotImage(img_arry, label):

    for im, l in zip(img_arry,label):
        plt.figure(figsize=(5,5))
        plt.imshow(im)
        plt.show()



In [ ]:
plotImage(t_img[:5], label[:5])

# **Building Model**

In [ ]:
from keras.layers import Dense,Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
import keras

In [ ]:
base_model = VGG19(input_shape=(256,256,3), include_top=False)

In [ ]:
for layer in base_model.layers:
    layer.trainable=False

In [ ]:
base_model.summary()

In [ ]:
x=Flatten()(base_model.output)
x = Dense(units=10, activation='softmax')(x)

In [ ]:
#Creating model

model= Model(base_model.input,x)

In [ ]:
model.summary()

# **Early Stopping And Model Check Point**

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [ ]:
early_stopping =EarlyStopping(monitor="val_accuracy", min_delta=0.01,patience=3, verbose=1)
#model checkpoint

mc= ModelCheckpoint(filepath="model.h5",monitor='val_accuracy',min_delta=0.01,patience=3, verbose=1, save_best_only=True)


cb =[early_stopping,mc]

In [ ]:
model.compile(optimizer='adam',loss=keras.losses.categorical_crossentropy,metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train,steps_per_epoch=16,epochs=20,verbose=1,callbacks=cb,validation_data=validation,validation_steps=16) 

In [ ]:
h = history .history
h.keys()

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c='red')
plt.title('acc vs val_acc')
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_accuracy'], c='red')
plt.title('acc vs val_acc')
plt.show()

In [ ]:
from keras.models import load_model

In [ ]:
# load best model
model=load_model("/content/model.h5")



In [ ]:
acc=model.evaluate_generator(validation)[1]


In [ ]:
list(train.class_indices.values()), list(train.class_indices.keys())

In [ ]:
ref=dict(zip(list(train.class_indices.values()), list(train.class_indices.keys()) ))

In [ ]:
def prediction(path):
    img=load_img(path,target_size=(256,256))
    i=img_to_array(img)
    im=preprocess_input(i)
    img=np.expand_dims(im,axis=0)
    pred = np.argmax(model.predict(img))
    print(f" This image belongs to { ref[pred] } ")

In [ ]:
path="/content/drive/MyDrive/NewPlantDiseasesDataset(Augmented)/Test/23857b36-24f3-4745-a877-dedde5c5931c___YLCV_NREC 2520.JPG"
prediction(path)

# **Thank You**